<a href="https://colab.research.google.com/github/ynyonato/HandsOn/blob/updates/ia_me_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

In [ ]:
# Télécharger stopwords et autres ressources NLTK si besoin
nltk.download('stopwords')
nltk.download('punkt')

In [5]:
# Chargement des données (ici on suppose que le fichier csv est uploadé dans Colab)
df = pd.read_csv('ia-me/suivi_activites.csv', sep=';', encoding='latin-1', quotechar='"')

# Apercu
print(df.columns)
print(df.head())
print(df.info())

Index(['ID_Activite', 'Date', 'Type_Activite', 'Nombre_Participants',
       'Localisation', 'Feedback', 'Resultat_Attendu'],
      dtype='object')
   ID_Activite        Date             Type_Activite  Nombre_Participants  \
0            1  05/01/2025  Distribution de matériel                   25   
1            2  10/01/2025         Atelier formation                   15   
2            3  15/01/2025          Suivi individuel                   10   
3            4  20/01/2025  Distribution de matériel                   30   
4            5  25/01/2025         Atelier formation                   20   

  Localisation                                      Feedback Resultat_Attendu  
0      Ville A    Les bénéficiaires ont apprécié la rapidité         Réussite  
1      Ville B                Manque de matériel pédagogique           Risque  
2      Ville A  Très bonne interaction avec les participants         Réussite  
3      Ville C  Certains participants sont arrivés en retard         

In [ ]:
# Conversion des données de la colonne Date en type date
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')

# Traitement des valeurs manquantes (voir nombre)
print(df.isnull().sum())

# Vérification des doublons
print(df.duplicated().sum())

# Suppression des doublons
df = df.drop_duplicates()

In [ ]:
# Nettoyage basique du texte (feedback)
def clean_text(text):
    if pd.isnull(text):
        return ""
    # Mise en minuscule
    text = text.lower()
    # Suppression ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Suppression chiffres
    text = re.sub(r'\d+', '', text)
    # Suppression des stopwords
    text = ' '.join([mot for mot in text.split() if mot not in stop_words])
    return text

df['Feedback_clean'] = df['Feedback'].apply(clean_text)

# Vérification si le traitement s'est bien déroulé et que la colonne traitée n'est pas vide
print(df['Feedback_clean'].head())

À ce stade :
*   La colonne Date est bien formatée
*   Les doublons sont supprimés
*   Élément de liste
*   Les Feedback sont nettoyés et une nouvelle colonne Feedback_clean est créée
*   Les champs manquants sont visibles pour traitement ultérieurÉlément de liste



In [ ]:
# ÉTAPE 2 : ANALYSE EXPLORATOIRE DE BASE (EDA)

# Aperçu des statistiques descriptives des colonnes numériques
print("Résumé statistique des colonnes numériques :")
print(df.describe())  # Affiche count, mean, std, min, 25%, 50%, 75%, max pour les colonnes numériques

# Nombre de valeurs uniques par colonne
print("\nNombre de valeurs uniques par colonne :")
print(df.nunique())  # Permet d'identifier les variables catégorielles ou constantes

# Aperçu rapide des types de données
print("\nTypes de données dans le DataFrame :")
print(df.dtypes)  # Vérifie les types (object, int64, float64, etc.)

# Vérification de la présence de valeurs manquantes
print("\nNombre de valeurs manquantes par colonne :")
print(df.isnull().sum())  # Pour détecter les colonnes incomplètes ou problématiques

In [ ]:
# ÉTAPE 3 : NETTOYAGE ET PRÉPARATION DES DONNÉES

# 1. Suppression des doublons
df = df.drop_duplicates()
print("✔️ Doublons supprimés.")

# 2. Suppression des colonnes inutiles (à adapter si tu identifies des colonnes non pertinentes)
# Exemple : si une colonne 'ID' ou 'timestamp' est sans intérêt
# df = df.drop(['NomDeLaColonne1', 'NomDeLaColonne2'], axis=1)

# 3. Gestion des valeurs manquantes (ici on les supprime, on peut aussi les remplir si besoin)
df = df.dropna()
print("✔️ Lignes avec valeurs manquantes supprimées.")

# 4. Harmonisation des noms de colonnes (sans espaces, minuscules, underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("✔️ Noms de colonnes nettoyés :", df.columns.tolist())

# 5. Conversion de colonnes en types appropriés (exemple : dates ou numériques)
# Exemple : convertir une colonne 'date' en datetime
# df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Exemple : convertir une colonne en numérique
# df['revenu'] = pd.to_numeric(df['revenu'], errors='coerce')

# 6. Affichage de l’état final du jeu de données
print("\n✔️ Données prêtes. Aperçu des premières lignes :")
print(df.head())

In [ ]:
# Étape 4 : Visualisation des données avec matplotlib et seaborn

import matplotlib.pyplot as plt
import seaborn as sns

# Configuration de base pour un style clair
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# 1. Histogramme pour une variable numérique
# Exemple : 'revenu' – à adapter selon tes colonnes
# Remplace 'revenu' par le nom réel de ta colonne
if 'revenu' in df.columns:
    sns.histplot(data=df, x='revenu', kde=True, color='skyblue')
    plt.title('Distribution des revenus')
    plt.xlabel('Revenu')
    plt.ylabel('Fréquence')
    plt.show()

# 2. Diagramme en barres pour une variable catégorielle
# Exemple : 'produit'
if 'produit' in df.columns:
    sns.countplot(data=df, x='produit', palette='Set2')
    plt.title('Nombre d\'occurrences par produit')
    plt.xticks(rotation=45)
    plt.ylabel('Nombre')
    plt.show()

# 3. Boîte à moustaches (boxplot) pour voir les outliers
# Exemple : 'revenu' par 'produit'
if 'revenu' in df.columns and 'produit' in df.columns:
    sns.boxplot(data=df, x='produit', y='revenu', palette='Set3')
    plt.title('Répartition des revenus par produit')
    plt.xticks(rotation=45)
    plt.show()

# 4. Matrice de corrélation (pour les variables numériques)
corr_matrix = df.select_dtypes(include='number').corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matrice de corrélation')
plt.show()

In [ ]:
# Étape 5 : Préparation des données pour le ML

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Sélection des colonnes pertinentes
# À adapter selon tes colonnes — exemple ici
colonnes_utiles = ['revenu', 'age', 'produit', 'sexe', 'achat']  # à modifier
df_ml = df[colonnes_utiles].copy()

# 2. Encodage des variables catégorielles
# Exemple : encodage de 'produit' et 'sexe' si elles sont catégorielles
cat_cols = df_ml.select_dtypes(include='object').columns

# Utilisation de LabelEncoder pour transformer les catégories en nombres
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_ml[col] = le.fit_transform(df_ml[col])
    label_encoders[col] = le

# 3. Séparation X (features) et y (target)
# Supposons que la colonne 'achat' est la cible (binaire : 0 ou 1)
X = df_ml.drop('achat', axis=1)
y = df_ml['achat']

# 4. Séparation en ensemble d'entraînement et de test (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Standardisation des données numériques
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Étape 6 : Entraînement d’un modèle de Machine Learning

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Initialiser le modèle avec des hyperparamètres de base
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Entraînement du modèle sur l'ensemble d'entraînement
model.fit(X_train_scaled, y_train)

# 3. Prédiction sur les données de test
y_pred = model.predict(X_test_scaled)

# 4. Évaluation du modèle
# a. Taux de bonne classification (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy : {accuracy:.2%}")

# b. Matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred))

# c. Rapport de classification détaillé (précision, rappel, F1-score)
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))


In [ ]:
# Suppression des stopwords
stop_words = set(stopwords.words('french'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['Feedback_clean'] = df['Feedback_clean'].apply(remove_stopwords)

In [ ]:
# Check if there are any non-empty strings after cleaning and stopword removal
if df['Feedback_clean'].str.strip().eq('').all():
    print("Erreur: Après nettoyage et suppression des mots vides, la colonne 'Feedback_clean' est vide. Impossible de créer un vocabulaire TF-IDF.")
    # Option 1: Print some raw feedback to understand why it's empty
    print("\nExemples de feedback originaux :")
    print(df['Feedback'].head())
    # Option 2: You might need to revisit your cleaning steps or data source
else:
    vectorizer = TfidfVectorizer(max_features=20)
    X = vectorizer.fit_transform(df['Feedback_clean'])
    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

    # Somme des TF-IDF par mot pour voir les plus importants
    tfidf_sum = tfidf_df.sum().sort_values(ascending=False)
    print("Top mots-clés TF-IDF :")
    print(tfidf_sum)

In [ ]:
X = vectorizer.fit_transform(df['Feedback_clean'])
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Somme des TF-IDF par mot pour voir les plus importants
tfidf_sum = tfidf_df.sum().sort_values(ascending=False)
print("Top mots-clés TF-IDF :")
print(tfidf_sum)

In [ ]:
# Analyse de sentiment avec TextBlob (en français)
# On utilise TextBlob directement ici, qui marche mieux en anglais, mais ça donnera un aperçu
# Pour un vrai traitement en français, on pourrait utiliser d’autres libs (ex: HuggingFace)
def sentiment_polarity(text):
    return TextBlob(text).sentiment.polarity

df['Sentiment'] = df['Feedback_clean'].apply(sentiment_polarity)

In [ ]:
# Catégorisation simple du sentiment
def categorize_sentiment(score):
    if score > 0.1:
        return 'Positif'
    elif score < -0.1:
        return 'Négatif'
    else:
        return 'Neutre'

df['Sentiment_cat'] = df['Sentiment'].apply(categorize_sentiment)

In [ ]:
# Affichage des counts sentiment
print("\nRépartition des sentiments :")
print(df['Sentiment_cat'].value_counts())

In [ ]:
# Nuage de mots pour feedback global
all_text = ' '.join(df['Feedback_clean'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

plt.figure(figsize=(12,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Nuage de mots - Feedback")
plt.show()

In [ ]:
# Graphique des sentiments par type d'activité
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='Type_Activite', hue='Sentiment_cat')
plt.title("Sentiments par Type d'Activité")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Moyenne de sentiment par localisation
sentiment_localisation = df.groupby('Localisation')['Sentiment'].mean().sort_values()

plt.figure(figsize=(8,5))
sentiment_localisation.plot(kind='bar', color='skyblue')
plt.title("Moyenne du sentiment par localisation")
plt.ylabel("Score moyen de sentiment")
plt.show()

In [ ]:
# Croisement nombre participants vs sentiment (boxplot)
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='Sentiment_cat', y='Nombre_Participants')
plt.title("Nombre de participants selon sentiment")
plt.show()